### 본 notebook에서 시도한 augmentation
- KoEDA augmentation
- remove stopwords in question
- multi-answer data augmentation|

In [16]:
import os
import json

import re

path = 'drive/MyDrive/Goorm_Project/Project2/Data/'

Josa in Label

In [ ]:
### label에 조사 있는지 파악 ###
from konlpy.tag import Twitter
 
twitter = Twitter()
twitter.pos('나는 밥을 먹는다')

count = 0
for i in range(len(dataset)):
  for word, tag in twitter.pos(dataset[i]['answers'][0]['text']):
    if tag == 'Josa':
      count += 1

print(f'조사가 포함된 answers 수: {count}')
print(f'전체 answer에서 조사 비율: {count/len(dataset)*100}%')

KoEDA Data augmentation

In [ ]:
### 발표용 KoEDA 예시 ###
with open(path + 'train.json', 'r', encoding='utf-8') as fd:
  data = json.load(fd)

from koeda import EDA

eda = EDA(morpheme_analyzer="Okt", alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, prob_rd=0.1)

data['data'].append(data['data'][0])
context = data['data'][0]['paragraphs'][0]['context']
answer = data['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['text']
new_context = eda(context, p=(0.1, 0.1, 0.1, 0.1), repetition=1)
answer = data['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['text']

print(context)
print(new_context)
print(answer)

answer in new_context

In [ ]:
# 최초 한 번만 실행
# KoEDA data 생성

with open(path + 'train.json', 'r', encoding='utf-8') as fd:
  data = json.load(fd)

import copy
from tqdm import tqdm
from koeda import EDA

eda = EDA(morpheme_analyzer="Okt", alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, prob_rd=0.1)

length = len(data['data']) 
print(f'원본 data 길이: {length}')
new_data = copy.deepcopy(data)

for idx in tqdm(range(length)):
  context = data['data'][idx]['paragraphs'][0]['context']
  answer = data['data'][idx]['paragraphs'][0]['qas'][0]['answers'][0]['text']

  while True:
    new_context = eda(context, p=(0.7, 0.7, 0.7, 0.7), repetition=1)
    if (answer in new_context) and (context != new_context):
      break
  
  new_data['data'][idx]['paragraphs'][0]['context'] = new_context

print('\n', f'augmentation data 길이: ', len(new_data['data']), '\n')
print(data['data'][0])
print(new_data['data'][0])

In [ ]:
# new_data의 answer_start 변경
for idx in range(len(new_data['data'])):
  context = new_data['data'][idx]['paragraphs'][0]['context']
  answer = new_data['data'][idx]['paragraphs'][0]['qas'][0]['answers'][0]['text']
  answer_start = context.find(answer)
  new_data['data'][idx]['paragraphs'][0]['qas'][0]['answers'][0]['answer_start'] = answer_start

print(data['data'][0]['paragraphs'][0]['context'])
print(data['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['answer_start'])
print(new_data['data'][0]['paragraphs'][0]['context'])
print(new_data['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['answer_start'])

In [ ]:
# 기존 data와 new_data 병합
length = len(new_data['data'])
for idx in range(length):
  data['data'].append(new_data['data'][idx])

print(len(data['data']))
print(data['data'][9789])

# 병합 파일 저장
with open(path + 'KoEDA_train.json', 'w') as new:
  json.dump(data, new)

Remove stopwords in Question

In [ ]:
# Question stopwords, 조사 제거
! pip install konlpy "tweepy<4.0.0"

from konlpy.tag import Okt
import re
import json

with open('drive/MyDrive/Goorm_Project/Project2/Data/KoEDA_train.json', 'r', encoding='utf-8') as fd:
    data = json.load(fd)

print(data['data'][0]['paragraphs'][0]['qas'][0]['question'])

# Question에서 조사 제거
stopwords = ['을', '를', '이', '가', '은', '는', 'null'] #불용어 설정
tokenizer = Okt() #형태소 분석기 

for idx in range(len(data['data'])):
  question = data['data'][idx]['paragraphs'][0]['qas'][0]['question']
  question = re.sub('[^A-Za-z0-9가-힣]', ' ', question)
  for s in stopwords:
    if s in question:
      question = re.sub(s, ' ', question)
  
  data['data'][idx]['paragraphs'][0]['qas'][0]['question'] = question
print(data['data'][0]['paragraphs'][0]['qas'][0]['question'])

with open('drive/MyDrive/Goorm_Project/Project2/Data/No_Stop_Answer_KoEDA_train.json', 'w') as fd2:
    json.dump(data, fd2)

Split validation set

In [ ]:
with open(path + 'train.json', 'r', encoding='utf-8') as fd:
    sample = json.load(fd)

In [ ]:
category_list = list()
for idx in range(len(sample['data'])):
  category = sample['data'][idx]['news_category']
  category_list.append(category)

category_set = set(category_list)

category_dict = dict()
for cat in category_set:
  count = category_list.count(cat)
  ratio = count / len(category_list) * 100
  category_dict[cat] = (count, ratio)

for k, v in category_dict.items():
  print(k, v)

중기과학 (59, 0.6027173357850648)
 (3, 0.030646644192460923)
로&비즈 (37, 0.3779752783736848)
IT모바일 (169, 1.7264276228419653)
전국 (107, 1.0930636428644396)
생활경제 (326, 3.3302686689140875)
책마을 (133, 1.3586678925324343)
산업섹션 (445, 4.54591888854837)
IT 과학 (80, 0.8172438451322914)
산업 (178, 1.8183675554193484)
기업&CEO (230, 2.3495760547553375)
자동차 (175, 1.7877209112268873)
교육산업 (50, 0.5107774032076821)
None (2510, 25.64102564102564)
미디어 (200, 2.0431096128307282)
금융 (231, 2.359791602819491)
테크 (158, 1.6140565941362754)
부동산 (425, 4.341607927265298)
기업&IT (65, 0.6640106241699867)
생활 (202, 2.0635407089590356)
문화/TV (355, 3.6265195627745426)
중기 의료 (73, 0.7457350086832158)
생생헬스 (14, 0.14301767289815098)
국제 (448, 4.576565532740831)
게임 (187, 1.9103074879967312)
JOB (57, 0.5822862396567575)
과학 (40, 0.4086219225661457)
사회 (549, 5.60833588722035)
중기 (116, 1.1850035754418224)
경제 (412, 4.208805802431301)
종합 (1414, 14.44478496271325)
경제금융 (49, 0.5005618551435285)
커버스토리 (29, 0.2962508938604556)
기획 (263, 2.686689140

Multi-Answer Data augmentation

In [19]:
with open(path + 'train.json', 'r', encoding='utf-8') as fd:
    sample = json.load(fd)

In [ ]:
from pprint import pprint

count_list = list()
count_11 = list()
count_7 = list()
count_6 = list()
count_5 = list()
count_4 = list()
count_3 = list()
count_2 = list()
count_dict = dict()
for idx in range(len(sample['data'])):
  cnt = len(sample['data'][idx]['paragraphs'][0]['qas'][0]['answers'])
  if cnt != 1:
    count_list.append(cnt)
    if cnt == 11:
      count_11.append(idx)
    if cnt == 7:
      count_7.append(idx)
    if cnt == 6:
      count_6.append(idx)
    if cnt == 5:
      count_5.append(idx)
    if cnt == 4:
      count_4.append(idx)
    if cnt == 3:
      count_3.append(idx)
    if cnt == 2:
      count_2.append(idx)
        

idx = 0
count_list_list = [count_2, count_3, count_4, count_5, count_6, count_7, count_11]
for i in set(count_list):
  count_dict[i] = tuple(count_list_list[idx])
  idx += 1


print(f'answer가 2개 이상인 data 수: {len(count_list)}')
print(f'answer 개수 목록: {set(count_list)}')
print(f'Mult-answer data 수: {len(count_2)*2 + len(count_3)*3 + len(count_4)*4 + len(count_5)*5 + len(count_6)*6 + len(count_7)*7 + len(count_11)*11}')
print(count_dict)

answer가 2개 이상인 data 수: 3629
answer 개수 목록: {2, 3, 4, 5, 6, 7, 11}
Mult-answer data 수: 8237
{2: (0, 3, 11, 13, 14, 18, 24, 28, 29, 32, 34, 38, 39, 45, 47, 50, 55, 57, 61, 62, 70, 72, 73, 75, 82, 83, 84, 86, 90, 93, 94, 96, 98, 99, 108, 109, 111, 112, 114, 115, 116, 117, 121, 122, 123, 125, 128, 130, 131, 134, 135, 137, 145, 147, 157, 158, 159, 160, 165, 169, 180, 183, 188, 191, 202, 203, 210, 211, 218, 220, 221, 226, 228, 231, 235, 238, 243, 245, 246, 247, 251, 252, 260, 269, 271, 272, 273, 275, 276, 280, 281, 283, 284, 288, 290, 294, 295, 296, 300, 307, 308, 309, 311, 315, 321, 324, 325, 327, 330, 331, 337, 338, 339, 340, 341, 343, 344, 346, 347, 353, 356, 358, 361, 363, 364, 366, 375, 383, 384, 389, 392, 393, 399, 400, 402, 404, 411, 413, 418, 421, 422, 423, 424, 435, 437, 439, 443, 452, 457, 459, 460, 464, 465, 467, 468, 470, 473, 474, 477, 484, 486, 488, 493, 496, 500, 506, 512, 513, 516, 521, 524, 530, 533, 534, 536, 537, 538, 539, 546, 552, 554, 557, 558, 560, 567, 568, 569, 570, 5

In [20]:
!pip install koeda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import copy
from tqdm import tqdm
from koeda import EDA

# SR(동의어)만 적용
multi_answer_eda = EDA(morpheme_analyzer="Okt", alpha_sr=0.9)

length = len(sample['data']) 
print(f'원본 data 길이: {length}')

count_list_list = [count_2, count_3, count_4, count_5, count_6, count_7, count_11]

multi_answer_data = copy.deepcopy(sample)
i = 0
for count_i in tqdm(count_list_list):
  zz = len(sample['data'][count_i[0]]['paragraphs'][0]['qas'][0]['answers'])
  print(f'\n→ answer 갯수 = {zz} generation 중 ....')

  for idx in count_i:
    answer_list = list() 
    context = sample['data'][idx]['paragraphs'][0]['context']
    for lens in range(len(sample['data'][idx]['paragraphs'][0]['qas'][0]['answers'])):
      answer_list.append(sample['data'][idx]['paragraphs'][0]['qas'][0]['answers'][lens]['text'])

    for ans in answer_list:
      while True:
        new_context = multi_answer_eda(context, p=(0.9, 0, 0, 0), repetition=1)
        if (ans in new_context) and (context != new_context):
          break

      multi_answer_data['data'][i]['news_category'] = sample['data'][idx]['news_category']
      multi_answer_data['data'][i]['paragraphs'][0]['qas'][0]['question'] = sample['data'][idx]['paragraphs'][0]['qas'][0]['question']
      multi_answer_data['data'][i]['paragraphs'][0]['context'] = new_context
      multi_answer_data['data'][i]['paragraphs'][0]['qas'][0]['answers'][0]['text'] = ans
      i += 1

원본 data 길이: 9789


  0%|          | 0/7 [00:00<?, ?it/s]


→ answer 갯수 = 2 generation 중 ....


 14%|█▍        | 1/7 [15:43<1:34:18, 943.15s/it]


→ answer 갯수 = 3 generation 중 ....


...

 71%|███████▏  | 5/7 [22:01<04:40, 140.42s/it]


→ answer 갯수 = 7 generation 중 ....


 86%|████████▌ | 6/7 [22:05<01:34, 94.25s/it] 


→ answer 갯수 = 11 generation 중 ....


100%|██████████| 7/7 [22:07<00:00, 189.71s/it]


In [ ]:
# Multi answer data 수에 맞게 길이 조정
print(f'Multi-answer data 수: {len(count_2)*2 + len(count_3)*3 + len(count_4)*4 + len(count_5)*5 + len(count_6)*6 + len(count_7)*7 + len(count_11)*11}')
multi_answer_data['data'] = multi_answer_data['data'][:8237]
print(len(multi_answer_data['data']))

Multi-answer data 수: 8237
8237


In [ ]:
# multi_answer_data의 answer_start 변경
for idx in range(len(multi_answer_data['data'])):
  context = multi_answer_data['data'][idx]['paragraphs'][0]['context']
  answer = multi_answer_data['data'][idx]['paragraphs'][0]['qas'][0]['answers'][0]['text']
  answer_start = context.find(answer)
  multi_answer_data['data'][idx]['paragraphs'][0]['qas'][0]['answers'][0]['answer_start'] = answer_start

In [ ]:
pprint(multi_answer_data['data'][0]) 
pprint(multi_answer_data['data'][1]) 

{'news_category': '종합',
 'paragraphs': [{'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 '
                            '이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 '
                            '장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 '
                            '100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 '
                            '하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 '
                            '만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 '
                            '내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 '
                            '20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 '
                            '장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘 '
                            '작가 가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 '
                            '이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 '
                            '평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, '
                            '강

In [ ]:
# 기존 data와 new_data 병합
length = len(multi_answer_data['data'])
for idx in range(length):
  sample['data'].append(multi_answer_data['data'][idx])

print(len(sample['data']))

# 병합 파일 저장
with open(path + 'multi_answer_train.json', 'w') as newest:
  json.dump(sample, newest)

18026


Multi-Answer + KoEDA

In [35]:
with open(path + 'train.json', 'r', encoding='utf-8') as fd3:
    sample = json.load(fd3)

with open(path + 'KoEDA_train.json', 'r', encoding='utf-8') as fd:
    KoEDA_sample = json.load(fd)

with open(path + 'multi_answer_train_.json', 'r', encoding='utf-8') as fd2:
    multi_answer_sample = json.load(fd2)

In [45]:
only_multi_answer_sample = copy.deepcopy(multi_answer_sample)
only_multi_answer_sample['data'] = only_multi_answer_sample['data'][9789:]

# KoEDA data와 only_multi_answer data 병합
length = len(only_multi_answer_sample['data'])
for idx in range(length):
  KoEDA_sample['data'].append(only_multi_answer_sample['data'][idx])

print(len(KoEDA_sample['data']))

# 병합 파일 저장
with open(path + 'KoEDA_Multi_answer.json', 'w') as hi:
  json.dump(KoEDA_sample, hi)

27815
